### evaluate pipeline & exp log

- baseline vs. new model
    - qwen2.5-7B-Instruct on gsm8k test dataset
    - report 85.4%;
        - https://arxiv.org/pdf/2412.15115
- new model 训练用数据集，及超参等；
- evaluate metrics
    - accuracy
- 实验前，列好表，留好空，跑实验就是填空的过程；

### inference test

- `Qwen/Qwen2.5-7B-Instruct` on `gsm8k` test set, on a dual 4090s:

```
# 单卡
python vllm_tp_dp.py --mode dp --num_gpus 1
# dp = 2
python vllm_tp_dp.py --mode dp --num_gpus 2
# tp = 2
python vllm_tp_dp.py --mode tp --num_gpus 2

# 单卡
python sglang_tp_dp.py --mode dp --num_gpus 1
# dp = 2
python sglang_tp_dp.py --mode dp --num_gpus 2
# tp = 2
python sglang_tp_dp.py --mode tp --num_gpus 2

```

|        |      | time(s) | accuracy |
|--------|------|------|----------|
| vllm   | 单卡  |   115.24   |  1034.0/1319 = 0.7839  |
| vllm   | dp=2 |   80.88    |  1031.0/1319=0.7817  |
| vllm   | tp=2 |   132.45   | 1034.0/1319 = 0.7839 |
| sglang | 单卡  |   120.17   | 1062.0/1319 = 0.8051 |
| sglang | dp=2 |   82.96    | 1069.0/1319 = 0.8105 |
| sglang | tp=2 |   91.41    | 1058.0/1319 = 0.8021 |

- dp & tp
    - dp size: 就是模型复制的次数（model replicas）
    - num_gpus = dp_size x tp_size
- vllm
    - https://docs.vllm.ai/en/latest/serving/offline_inference.html
    - dual 4090s (不支持 p2p access)
        - `# GPU blocks: 25626, # CPU blocks: 9362`
        ```
        INFO 03-21 20:07:55 distributed_gpu_executor.py:57] # GPU blocks: 25626, # CPU blocks: 9362
        INFO 03-21 20:07:55 distributed_gpu_executor.py:61] Maximum concurrency for 32768 tokens per request: 12.51x-
        ```
        - 25626 * 16 / 32768 = 12.51x
- sglang
    - `python3 -m sglang.check_env`
    - https://docs.sglang.ai/backend/offline_engine_api.html
    - dp
        - sglang::scheduler_DP0_TP0
        - sglang::scheduler_DP1_TP0
    - tp
        - sglang::scheduler_TP0
        - sglang::scheduler_TP1